# NLP With Deep Learning (W266)

Submission by *Carolina Arriaga, Ayman, Abhi Sharma*

Winter 2021 | UC Berkeley

## Notebook Overview

This notebook contains the list of metrics authored by the team to evaluate summarization models

References list:

https://arxiv.org/pdf/2007.12626.pdf

https://github.com/Yale-LILY/SummEval

# Required Installs

In [1]:
!pip install -q datasets 
!pip install dask
!git clone https://github.com/huggingface/datasets.git
!pip install -q transformers
!pip install -q sentencepiece
!pip install scikit-learn==1.0.1
!pip install folium==0.2.1

# summeval
!pip install summ-eval

# rouge scorer
!pip install -q rouge_score
from datasets import load_metric

# module to calculate regular statistics
import statistics
# pretty print
import pprint
pp = pprint.PrettyPrinter(indent=4, depth=5)

# this is the original directory where we start in collab
original_dir = '/content'

     |████████████████████████████████| 298 kB 4.3 MB/s 
     |████████████████████████████████| 243 kB 59.4 MB/s 
     |████████████████████████████████| 61 kB 395 kB/s 
     |████████████████████████████████| 1.1 MB 71.2 MB/s 
     |████████████████████████████████| 132 kB 98.5 MB/s 
     |████████████████████████████████| 271 kB 54.7 MB/s 
     |████████████████████████████████| 192 kB 94.6 MB/s 
     |████████████████████████████████| 160 kB 85.9 MB/s 
Cloning into 'datasets'...
remote: Enumerating objects: 48460, done.
remote: Counting objects: 100% (4602/4602), done.
remote: Compressing objects: 100% (2037/2037), done.
remote: Total 48460 (delta 2384), reused 3968 (delta 2030), pack-reused 43858
Receiving objects: 100% (48460/48460), 67.44 MiB | 14.46 MiB/s, done.
Resolving deltas: 100% (21267/21267), done.
     |████████████████████████████████| 3.1 MB 3.6 MB/s 
     |████████████████████████████████| 3.3 MB 23.6 MB/s 
     |████████████████████████████████| 895 kB 92.7 MB/s 
  

In [2]:
# install datasets' dependencies
# TODO - setup.py doesn't work without required args - do we need this at all?
%cd /content/datasets
!pip install -q -e .
!python setup.py

/content/datasets
usage: setup.py [global_opts] cmd1 [cmd1_opts] [cmd2 [cmd2_opts] ...]
   or: setup.py --help [cmd1 cmd2 ...]
   or: setup.py --help-commands
   or: setup.py cmd --help

error: no commands supplied


In [3]:
# the packages require Pytorch(1.0+) and TensorFlow (2.2+)
import tensorflow as tf
print(tf.__version__)
import torch 
print(torch.__version__)

2.7.0
1.10.0+cu111


In [4]:
# load cnn-dm
from datasets import load_dataset
from datasets import get_dataset_config_names

def get_cnn_dm(version='3.0.0', split='test'):
  # configs = get_dataset_config_names("cnn_dailymail")
  # print(configs)
  return load_dataset('cnn_dailymail', version=version, split=split)
  
dataset = get_cnn_dm(version='3.0.0', split='validation')
# see dataset info
# pp.pprint(dataset.info)

Downloading:   0%|          | 0.00/3.51k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

Using custom data configuration default


  0%|          | 0/5 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/159M [00:00<?, ?B/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/default/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234. Subsequent calls will reuse this data.


# Get Sample Summary Predictions

We are now loading a pretrain model to get test predictions on the CNN/Daily dataset. We will then use these predictions to test our metric outputs. 

In [5]:
# import a tokenizer associated with the model
from transformers import PegasusTokenizer, TFPegasusForConditionalGeneration

def get_cnndm_by_id(dataset, id, return_article_only=True):
  id = id.replace('dm-test-', '')
  id = id.replace('dm-train-', '')
  id = id.replace('dm-dev-', '')
  id = id.replace('dm-validation-', '')
  id = id.replace('dm-val-', '')

  id = id.replace('cnn-test-', '')
  id = id.replace('cnn-train-', '')
  id = id.replace('cnn-dev-', '')
  id = id.replace('cnn-val-', '')
  id = id.replace('cnn-validation-', '')
  try:
    highlight = dataset.filter(lambda x: x['id'] == id)['highlights'][0]
    article = dataset.filter(lambda x: x['id'] == id)['article'][0]
  except:
    return None
  if return_article_only:
    return article
    
  return article, highlight
  
def get_sample_summaries(dataset, topN=2, model_type='pegasus', data_type='cnndm'):
  if model_type == 'pegasus' and data_type == 'cnndm':
    # articles to summarize
    articles = dataset[0:topN]['article']
    # references (golden)
    refs = dataset[0:topN]['highlights']
    model = TFPegasusForConditionalGeneration.from_pretrained('google/pegasus-xsum')
    tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-xsum')
    batch = tokenizer(articles, truncation=True, padding='longest', return_tensors="tf")
    translated = model.generate(**batch,  return_dict_in_generate=True, num_beams=5, top_k=3, min_length=50)
    predictions = tokenizer.batch_decode(translated[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)  
    return articles, refs, predictions
  else:
    raise Error("Model not defined")

In [6]:
articles, refs, predictions = get_sample_summaries(dataset, topN=5)

Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFPegasusForConditionalGeneration.

All the layers of TFPegasusForConditionalGeneration were initialized from the model checkpoint at google/pegasus-xsum.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFPegasusForConditionalGeneration for predictions without further training.


Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.36M [00:00<?, ?B/s]

In [7]:
for r in refs:
  print(r)
  print()

Accident happens in Santa Ynez, California, near where Crosby lives .
The jogger suffered multiple fractures; his injuries are not believed to be life-threatening .

Sigma Alpha Epsilon is being tossed out by the University of Oklahoma .
It's also run afoul of officials at Yale, Stanford and Johns Hopkins in recent months .

Religion professor Candida Moss appears in each episode of the program .
Moss was part of the original study to determine if relics found in Bulgaria could be the bones of John the Baptist.

Two police officers were shot Wednesday in Ferguson .
Hank Johnson, Michael Shank: Policing style needs rethink .

Clinton Foundation has taken money from foreign governments .
Bill Clinton:  "I believe we have done a lot more good than harm"



In [8]:
for p in predictions:
  print(p)
  print()

Singer-songwriter David Crosby was airlifted to a hospital after he hit a jogger with his car in California on Sunday evening, the California Highway Patrol said. (CNN) Singer-songwriter David Crosby hit a jogger with his car on Sunday evening, the California Highway Patrol said.

Sigma Alpha Epsilon (SAE) is one of the oldest fraternities in the United States, founded in 1856 at the University of Alabama but has faced a series of controversies in recent years, including hazing, alcohol abuse, sexual misconduct and a racist chant.

As the 40th episode of "Finding Jesus" airs, host Candida Moss answers your questions about the series about the life and times of Jesus, as well as those of the experts who have contributed to it over the past 40 days and days.

In the wake of the shooting of two police officers in Ferguson, Missouri, and the resignation of the town's police chief, here's what we need to know if we are to stem the tide of anger and cynicism that is engulfing America.

Forme

# Evaluation Metrics

Now we have created N example predictions, we can run a test between the prediction and the golden summary.

We are considering 4 dimensions along which we will evaluate summaries - coherence, consistency, relevance and fluency. These dimensions are derived from the SummEval paper.

To measure these dimensions, we first introduce several metrics that are correlated against them. We then perform an in-depth analysis of the contributions of each of these metrics against the said dimensions. 

## ROUGE

Some explanation on ROUGE
https://towardsdatascience.com/the-ultimate-performance-metric-in-nlp-111df6c64460

ROUGE-n recall=40% means that 40% of the n-grams in the reference summary are also present in the generated summary.

ROUGE-n precision=40% means that 40% of the n-grams in the generated summary are also present in the reference summary.

In [9]:
%cd /content

# needed for summeval
import os
os.environ["ROUGE_HOME"] = '/usr/local/lib/python3.7/dist-packages/summ_eval/ROUGE-1.5.5/'
import nltk
nltk.download('stopwords')
!pip install -U git+https://github.com/bheinzerling/pyrouge.git

/content
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
  Cloning https://github.com/bheinzerling/pyrouge.git to /tmp/pip-req-build-bphxb528
  Running command git clone -q https://github.com/bheinzerling/pyrouge.git /tmp/pip-req-build-bphxb528
  Created wheel for pyrouge: filename=pyrouge-0.1.3-py3-none-any.whl size=191923 sha256=8eb55c9a994be41c03d6c01589d9b20e10bbcade894a3c7ad32e54c941177c2f
  Stored in directory: /tmp/pip-ephem-wheel-cache-fqu_snrh/wheels/2f/1c/55/4e264b4e45f2a6f39a50e95cff45e57b54f202db830f125bb2
Successfully built pyrouge


### HuggingFace API

In [10]:
import datasets
def rouge_scores_hf(predictions, refs, use_stemmer=True, use_agregator=True, verbose_schema=False):
  rouge = datasets.load_metric('rouge')
  results = rouge.compute(predictions = predictions, references = refs, use_stemmer = use_stemmer, use_agregator = use_agregator)
  if verbose_schema:
    pp.pprint(rouge.inputs_description)
    pp.pprint(list(results.keys()))
  return results

In [11]:
r = rouge_scores_hf(predictions, refs)
r

Downloading:   0%|          | 0.00/2.17k [00:00<?, ?B/s]

{'rouge1': AggregateScore(low=Score(precision=0.17566871955142935, recall=0.3291666666666667, fmeasure=0.23122807017543862), mid=Score(precision=0.226698963144602, recall=0.4488095238095238, fmeasure=0.29518411413148254), high=Score(precision=0.27873171300343885, recall=0.6031398809523809, fmeasure=0.36139194139194136)),
 'rouge2': AggregateScore(low=Score(precision=0.03834373227453205, recall=0.058750000000000004, fmeasure=0.04562798092209856), mid=Score(precision=0.08477230370310349, recall=0.185787037037037, fmeasure=0.11368605714714393), high=Score(precision=0.12970383275261324, recall=0.32875, fmeasure=0.18348457350272232)),
 'rougeL': AggregateScore(low=Score(precision=0.14041272175751449, recall=0.24034902597402596, fmeasure=0.17904761904761904), mid=Score(precision=0.18008849221131698, recall=0.36682900432900434, fmeasure=0.23619047619047615), high=Score(precision=0.22102931747974588, recall=0.5013528138528139, fmeasure=0.29428571428571426)),
 'rougeLsum': AggregateScore(low=Sc

### ROUGE Recall Weighted Score

A summary's content can be evaluated based on the overlap of n-grams between the predicted summary and the gold sentence. We get this in the form of rouge scores.

We propose consolidating the metric Rouge in two ways. First, at a sentence level. Then, at a summary level.

**TODO: confirm that 'mid' in rouge results means 'avg'**

We proceed to calculate the mid recall for rouge at a sentence level for 1,2 and L configurations. Then, we extract the LSum mid recall score. We also provide a weighting scheme to prefer summary level vs sentence level rouge.

In [12]:
import math
def get_mid_recall_rouge_stats(agg_rouge_result, weights=[0.5,0.5], report_all_stats=True, verbose=True):
  assert len(weights) == 2
  assert float(sum(weights)) == 1.0
  assert 'rouge1' in agg_rouge_result
  assert 'rouge2' in agg_rouge_result
  assert 'rougeL' in agg_rouge_result
  assert 'rougeLsum' in agg_rouge_result

  mean_rouge_sentences = statistics.mean([agg_rouge_result['rouge1'].mid.recall, 
                  agg_rouge_result['rouge2'].mid.recall, 
                  agg_rouge_result['rougeL'].mid.recall])

  if report_all_stats:
    median_rouge_sentences = statistics.median([agg_rouge_result['rouge1'].mid.recall, 
                  agg_rouge_result['rouge2'].mid.recall, 
                  agg_rouge_result['rougeL'].mid.recall])
    
    min_rouge_sentences = min([agg_rouge_result['rouge1'].mid.recall, 
                  agg_rouge_result['rouge2'].mid.recall, 
                  agg_rouge_result['rougeL'].mid.recall])
    
    max_rouge_sentences = max([agg_rouge_result['rouge1'].mid.recall, 
                  agg_rouge_result['rouge2'].mid.recall, 
                  agg_rouge_result['rougeL'].mid.recall])

  mean_rouge_summary = agg_rouge_result['rougeLsum'].mid.recall

  overall_weighted_rouge = weights[0]*mean_rouge_sentences + weights[1]*mean_rouge_summary

  if verbose:
    print("Mean rouge (1,2,L) = {}".format(mean_rouge_sentences))
    print("Mean rouge (LSum) = {}".format(mean_rouge_summary))
    print("Mean w1*R_bar(1,2,L) + w2*R(LSum) = {}".format(overall_weighted_rouge))

  if report_all_stats:
    return mean_rouge_sentences, mean_rouge_summary, overall_weighted_rouge, median_rouge_sentences, min_rouge_sentences, max_rouge_sentences
  else:
    return mean_rouge_sentences, mean_rouge_summary, overall_weighted_rouge

In [13]:
get_mid_recall_rouge_stats(r)

Mean rouge (1,2,L) = 0.3338085217251884
Mean rouge (LSum) = 0.4058982683982684
Mean w1*R_bar(1,2,L) + w2*R(LSum) = 0.36985339506172843


(0.3338085217251884,
 0.4058982683982684,
 0.36985339506172843,
 0.36682900432900434,
 0.185787037037037,
 0.4488095238095238)

### Higher Order ROUGE

Next, we find a way to calculate ROUGE-3 and ROUGE-4 metrics since those are not traditionally available in the huggingface ROUGE API: https://github.com/huggingface/datasets/blob/master/metrics/rouge/rouge.py

We will calculate this in 2 ways - using the traditional exact match of n-grams and using a soft match of n-grams.

**Traditional method:**
https://github.com/danieldeutsch/sacrerouge/blob/master/doc/tutorials/evaluating-models.md

**Softmatch method:** 

Calculating ROUGE for trigrams or higher is traditionally harder because one is unlikely to find exact trigram or 4/5 gram matches between the reference and the prediction. Thus, we use a modified approach here.

We calculate the ROUGE Word Embedding (WE) score, which allows us to calculate ROUGE scores but with word embeddings in mind. Thus, bigrams like "the home" and "the house" may not match in traditional ROUGE calculations, but will be matched in ROUGE-we calculations.  

#### Exact Match

In [14]:
%cd /content
!pip install git+https://github.com/tagucci/pythonrouge.git
!apt-get install -y cpanminus
!cpanm --force XML::Parser

/content
  Cloning https://github.com/tagucci/pythonrouge.git to /tmp/pip-req-build-n2ldwnj0
  Running command git clone -q https://github.com/tagucci/pythonrouge.git /tmp/pip-req-build-n2ldwnj0
  Created wheel for pythonrouge: filename=pythonrouge-0.2-py3-none-any.whl size=285411 sha256=71e8e275754422c0da7dc0b026212db70ffe2e9d2da57461dff9f0edb9ed71f5
  Stored in directory: /tmp/pip-ephem-wheel-cache-_p30mhdt/wheels/f2/28/1a/47e2c73d5e7d1d49296a991e19d774c97e4c2e98dbf027a515
Successfully built pythonrouge
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libalgorithm-c3-perl libb-hooks-endofscope-perl libb-hooks-op-check-perl
  libclass-c3-perl libclass-c3-xs-perl libclass-method-modifiers-perl
  libclass-xsaccessor-perl libcpan-changes-perl libcpan-distnameinfo-perl
  libcpan-meta-check-perl libdata-optlist-perl libdata-perl-perl
  libdata-section-perl libdevel-callchecker-perl
  libde

In [15]:
from pythonrouge.pythonrouge import Pythonrouge 

# https://ilmoirfan.com/rouge-an-evaluation-metric-for-text-summarization/
def get_rouge(ngram, prediction, ref): 
  rouge = Pythonrouge(summary_file_exist=False,
                      summary=[[prediction]], reference=[[[ref]]],
                      n_gram=ngram, ROUGE_SU4=True, ROUGE_L=True,
                      ROUGE_W=True, ROUGE_W_Weight=1.2,
                      recall_only=False, stemming=True, stopwords=True,
                      word_level=True, length_limit=False,
                      use_cf=False, cf=95, scoring_formula='average',
                      resampling=True, samples=1000, favor=True, p=0.5)

  return rouge.calc_score()

In [16]:
get_rouge(ngram=4, prediction=predictions[0], ref=refs[0])

{'ROUGE-1-F': 0.14286,
 'ROUGE-1-P': 0.10714,
 'ROUGE-1-R': 0.21429,
 'ROUGE-2-F': 0.0,
 'ROUGE-2-P': 0.0,
 'ROUGE-2-R': 0.0,
 'ROUGE-3-F': 0.0,
 'ROUGE-3-P': 0.0,
 'ROUGE-3-R': 0.0,
 'ROUGE-4-F': 0.0,
 'ROUGE-4-P': 0.0,
 'ROUGE-4-R': 0.0,
 'ROUGE-L-F': 0.14286,
 'ROUGE-L-P': 0.10714,
 'ROUGE-L-R': 0.21429,
 'ROUGE-SU4-F': 0.03637,
 'ROUGE-SU4-P': 0.02632,
 'ROUGE-SU4-R': 0.05882,
 'ROUGE-W-1.2-F': 0.11097,
 'ROUGE-W-1.2-P': 0.09653,
 'ROUGE-W-1.2-R': 0.1305}

In [17]:
get_rouge(ngram=4, prediction="I like homes", ref=["I like homes"])

{'ROUGE-1-F': 1.0,
 'ROUGE-1-P': 1.0,
 'ROUGE-1-R': 1.0,
 'ROUGE-2-F': 0.0,
 'ROUGE-2-P': 0.0,
 'ROUGE-2-R': 0.0,
 'ROUGE-3-F': 0.0,
 'ROUGE-3-P': 0.0,
 'ROUGE-3-R': 0.0,
 'ROUGE-4-F': 0.0,
 'ROUGE-4-P': 0.0,
 'ROUGE-4-R': 0.0,
 'ROUGE-L-F': 1.0,
 'ROUGE-L-P': 1.0,
 'ROUGE-L-R': 1.0,
 'ROUGE-SU4-F': 0.0,
 'ROUGE-SU4-P': 0.0,
 'ROUGE-SU4-R': 0.0,
 'ROUGE-W-1.2-F': 1.0,
 'ROUGE-W-1.2-P': 1.0,
 'ROUGE-W-1.2-R': 1.0}

#### Soft Match (ROUGE-WE)

In [46]:
from summ_eval.rouge_we_metric import RougeWeMetric

# it takes some time to download embeddings, so be cautious when running this
global_we_rouge1 = RougeWeMetric(n_gram=1)
global_we_rouge2 = RougeWeMetric(n_gram=2)
global_we_rouge3 = RougeWeMetric(n_gram=3)
global_we_rouge4 = RougeWeMetric(n_gram=4)
global_we_rouge5 = RougeWeMetric(n_gram=5)

def rouge_we_metric(refs, predictions, ngram=3, rouge=None, aggregate=False):
  if rouge == None:
    rouge = RougeWeMetric(n_gram = ngram)
  rouge_dict = rouge.evaluate_batch(predictions, refs, aggregate = aggregate)
  return rouge_dict

In [47]:
rouge_we_metric(refs, predictions, rouge = global_we_rouge3)

[{'rouge_we_3_f': 0.060606060606060615,
  'rouge_we_3_p': 0.045454545454545456,
  'rouge_we_3_r': 0.09090909090909091},
 {'rouge_we_3_f': 0.27272727272727276,
  'rouge_we_3_p': 0.225,
  'rouge_we_3_r': 0.34615384615384615},
 {'rouge_we_3_f': 0.16666666666666666,
  'rouge_we_3_p': 0.14634146341463414,
  'rouge_we_3_r': 0.1935483870967742},
 {'rouge_we_3_f': 0.14285714285714285,
  'rouge_we_3_p': 0.09523809523809523,
  'rouge_we_3_r': 0.2857142857142857},
 {'rouge_we_3_f': 0.37837837837837834,
  'rouge_we_3_p': 0.2545454545454545,
  'rouge_we_3_r': 0.7368421052631579}]

In [48]:
rouge_we_metric(refs, predictions, rouge = global_we_rouge4)

[{'rouge_we_4_f': 0.09374999999999999,
  'rouge_we_4_p': 0.06976744186046512,
  'rouge_we_4_r': 0.14285714285714285},
 {'rouge_we_4_f': 0.21875,
  'rouge_we_4_p': 0.1794871794871795,
  'rouge_we_4_r': 0.28},
 {'rouge_we_4_f': 0.17142857142857143,
  'rouge_we_4_p': 0.15,
  'rouge_we_4_r': 0.2},
 {'rouge_we_4_f': 0.14814814814814817,
  'rouge_we_4_p': 0.0975609756097561,
  'rouge_we_4_r': 0.3076923076923077},
 {'rouge_we_4_f': 0.3333333333333333,
  'rouge_we_4_p': 0.2222222222222222,
  'rouge_we_4_r': 0.6666666666666666}]

In [49]:
rouge_we_metric(refs, predictions, rouge = global_we_rouge5)

[{'rouge_we_5_f': 0.16129032258064516,
  'rouge_we_5_p': 0.11904761904761904,
  'rouge_we_5_r': 0.25},
 {'rouge_we_5_f': 0.2580645161290323,
  'rouge_we_5_p': 0.21052631578947367,
  'rouge_we_5_r': 0.3333333333333333},
 {'rouge_we_5_f': 0.2647058823529412,
  'rouge_we_5_p': 0.23076923076923078,
  'rouge_we_5_r': 0.3103448275862069},
 {'rouge_we_5_f': 0.11538461538461538,
  'rouge_we_5_p': 0.075,
  'rouge_we_5_r': 0.25},
 {'rouge_we_5_f': 0.3142857142857143,
  'rouge_we_5_p': 0.20754716981132076,
  'rouge_we_5_r': 0.6470588235294118}]

In [50]:
rouge_we_metric(["I like homes"], ["I like houses"], rouge = global_we_rouge2)

[{'rouge_we_2_f': 1.0, 'rouge_we_2_p': 1.0, 'rouge_we_2_r': 1.0}]

## SLOR

Based on Kann et al, sentence level fluency can be scored via a referenceless method with higher correlation with human fluency. ROUGE-LM is a reference-based metric which is a natural extension of WPSLOR (WordPiece Syntactic Log-Odds Ratio).

### Sentencizer

Now we have our predictions and we want to evaluate the sentence probability and the unigram probability.

To do this, we must take into account that a predicted summary can be composed of multiple sentences. In order to evaluate the fluency, we need to process our output summary and make sure each output is a list of sentences.

We use spacy's sentencizer to do the work for us. 

In [17]:
from spacy.lang.en import English
from spacy.pipeline import Sentencizer

config = {"punct_chars": None}

def to_sentences(text, verbose=True):
  sentencizer = Sentencizer()
  nlp = English()
  nlp.add_pipe(nlp.create_pipe("sentencizer"))
  doc = nlp(text)
  if verbose:
    pp.pprint("Number of sentences: {}".format(len(list(doc.sents))))
  
  return list(doc.sents)

In [18]:
sample_doc_summary = """The intuition behind subtracting unigram logprobabilities is that a token which is rare on its
own (in contrast to being rare at a given position in
the sentence) should not bring down the sentence’s
rating. The normalization by sentence length is
necessary in order to not prefer shorter sentences
over equally fluent longer ones. """

to_sentences(sample_doc_summary)

'Number of sentences: 2'


[The intuition behind subtracting unigram logprobabilities is that a token which is rare on its
 own (in contrast to being rare at a given position in
 the sentence) should not bring down the sentence’s
 rating., The normalization by sentence length is
 necessary in order to not prefer shorter sentences
 over equally fluent longer ones.]

In [19]:
prediction_in_sentences = list()
for doc_index in range(len(predictions)):
  print("Prediction number {}".format(doc_index+1))
  prediction_in_sentences.append(to_sentences(predictions[doc_index]))

print("\nNumber of predictions {}".format(len(prediction_in_sentences)))

# 1st prediction has 2 sentences
for i in range(len(prediction_in_sentences[0])):
  print("\nFirst prediction, sentence number: {}".format(i+1))
  print(prediction_in_sentences[0][i])

Prediction number 1
'Number of sentences: 2'
Prediction number 2
'Number of sentences: 1'
Prediction number 3
'Number of sentences: 1'
Prediction number 4
'Number of sentences: 1'
Prediction number 5
'Number of sentences: 2'

Number of predictions 5

First prediction, sentence number: 1
Singer-songwriter David Crosby was airlifted to a hospital after he hit a jogger with his car in California on Sunday evening, the California Highway Patrol said. (

First prediction, sentence number: 2
CNN) Singer-songwriter David Crosby hit a jogger with his car on Sunday evening, the California Highway Patrol said.


### SLOR Scorer

Pegasus (default model used to generate predictions) is based on BERT, but this is not a LM we can use to obtain probabilities of the sentences. BERT/GPT works by masking and providing probabilities of the masked tokens given an embbeding (cloze task).

We will use LM-Scorer from Simonepri (https://github.com/simonepri/lm-scorer) to obtain the log probabilities of a sentence based on a LM.

Copying from their Colab (https://colab.research.google.com/github/simonepri/lm-scorer/blob/master/examples/lm_scorer.ipynb#scrollTo=ljwsZkjT7QIx)

In [20]:
%cd /content 
!git clone https://github.com/simonepri/lm-scorer
#!sleep 60
#!pip install --user poetry

%cd lm-scorer

!poetry install
!pip install transformers

/content
Cloning into 'lm-scorer'...
remote: Enumerating objects: 396, done.
remote: Counting objects: 100% (142/142), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 396 (delta 75), reused 113 (delta 53), pack-reused 254
Receiving objects: 100% (396/396), 4.69 MiB | 17.52 MiB/s, done.
Resolving deltas: 100% (203/203), done.
/content/lm-scorer
/bin/bash: poetry: command not found


In [21]:
import torch
from lm_scorer.models.auto import AutoLMScorer as LMScorer
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import numpy as np
import re
import pprint as pp
import statistics

def cloze_finalword(text, tokenizer=None, model=None):
  if tokenizer == None or model == None:
    gpt2_variant = get_slor_gpt2_variant()
    # Load pre-trained model (weights) - this takes the most time
    model = GPT2LMHeadModel.from_pretrained(gpt2_variant, output_hidden_states = True, output_attentions = True)
    model.eval()
    tokenizer = GPT2Tokenizer.from_pretrained(gpt2_variant)

  whole_text_encoding = tokenizer.encode(text)
	# Parse out the stem of the whole sentence (i.e., the part leading up to but not including the critical word)
  text_list = text.split()
  stem = ' '.join(text_list[:-1])
  stem_encoding = tokenizer.encode(stem)
	# cw_encoding is just the difference between whole_text_encoding and stem_encoding
	# note: this might not correspond exactly to the word itself
	# e.g., in 'Joe flicked the grasshopper', the difference between stem and whole text (i.e., the cw) is not 'grasshopper', but
	# instead it is ' grass','ho', and 'pper'. This is important when calculating the probability of that sequence.
  cw_encoding = whole_text_encoding[len(stem_encoding):]
	# print (cw_encoding)
	# print (whole_text_encoding)

	# Run the entire sentence through the model. Then go "back in time" to look at what the model predicted for each token, starting at the stem.
	# e.g., for 'Joe flicked the grasshopper', go back to when the model had just received 'Joe flicked the' and
	# find the probability for the next token being 'grass'. Then for 'Joe flicked the grass' find the probability that
	# the next token will be 'ho'. Then for 'Joe flicked the grassho' find the probability that the next token will be 'pper'.

	# Put the whole text encoding into a tensor, and get the model's comprehensive output
  tokens_tensor = torch.tensor([whole_text_encoding])
	
  with torch.no_grad():
    outputs = model(tokens_tensor)
    predictions = outputs[0]   

  logprobs = []
	# start at the stem and get downstream probabilities incrementally from the model(see above)
	# I should make the below code less awkward when I find the time
  start = -1-len(cw_encoding)
  for j in range(start,-1,1):
      # print (j)
      raw_output = []
      for i in predictions[-1][j]:
        raw_output.append(i.item())

      logprobs.append(np.log(softmax(raw_output)))
			
	# if the critical word is three tokens long, the raw_probabilities should look something like this:
	# [ [0.412, 0.001, ... ] ,[0.213, 0.004, ...], [0.002,0.001, 0.93 ...]]
	# Then for the i'th token we want to find its associated probability
	# this is just: raw_probabilities[i][token_index]
  conditional_probs = []
  for cw,prob in zip(cw_encoding,logprobs):
      # print (prob[cw])
      conditional_probs.append(prob[cw])
  # now that you have all the relevant probabilities, return their product.
  return np.exp(np.sum(conditional_probs))
 
def get_slor_gpt2_variant(variant='gpt2-medium'):
  # Available models
  # list(LMScorer.supported_model_names())
  # => ["gpt2", "gpt2-medium", "gpt2-large", "gpt2-xl", distilgpt2"]
  # gpt2-large will need higher memory, so be sure you're running on a beefy machine
  assert variant in ["gpt2", "gpt2-medium", "gpt2-large", "gpt2-xl", "distilgpt2"]
  return variant

def softmax(x):
	exps = np.exp(x)
	return np.divide(exps, np.sum(exps))
 
def slor(probability_sentence, log_probability_unigrams, text):
  return (np.log(probability_sentence) - np.sum(log_probability_unigrams))/len(text.split())

def get_lm_scorer(model):
  # Load model to cpu or cuda
  device = "cuda:0" if torch.cuda.is_available() else "cpu"
  batch_size = 1
  scorer = LMScorer.from_pretrained(model, device=device, batch_size=batch_size)
  return scorer

def mean_slor(list_of_slor):
  return statistics.mean(list_of_slor)

# main method for slor scoring of a summary
# reports slor scores for a list of sentences
# here, we need to break the summary output into multiple sentences with the sentencizer
def sentence_slor_scores(sentences, scorer=None, tokenizer=None, model=None, verbose=False):
  if scorer == None:
    model_variant = get_slor_gpt2_variant()
    scorer = get_lm_scorer(model_variant)

  slor_scores = []
  for sentence in sentences:
    # Compute sentence conditional prob
    sentence_score = cloze_finalword(sentence, tokenizer, model)
    
    # Compute sentence score as the product of tokens' probabilities
    log_unigram_probs_sentence = scorer.tokens_score(sentence, log=True)
    
    # Sentence Log Odds Ratio
    slor_score = slor(sentence_score, log_unigram_probs_sentence[0], sentence)

    if verbose:
      print("Sentence is: {}".format(sentence))
      print("Sentence score is: {}".format(sentence_score))
      print("Sentence log_score is: {}".format(np.log(sentence_score)))
      print("Unigram probs is: {}".format(log_unigram_probs_sentence[0]))
      print("Slor score is: {}".format(-slor_score))
      print()

    slor_scores.append(-slor_score)
  
  return slor_scores


In [22]:
import numpy as np

def slor_main(summaries, verbose=False):
  result = []
  gpt2_variant = get_slor_gpt2_variant()
  slor_scorer = get_lm_scorer(gpt2_variant)  

  # Load pre-trained model (weights) - this takes the most time
  model = GPT2LMHeadModel.from_pretrained(gpt2_variant, output_hidden_states = True, output_attentions = True)
  model.eval()
  tokenizer = GPT2Tokenizer.from_pretrained(gpt2_variant)
  
  for summ in summaries:
    try:
      sentences = to_sentences(summ, verbose = verbose)
      # spacy's sentencing makes the type diff from string
      sentences = [str(s) for s in sentences]
      # print(sentences)
      scores = sentence_slor_scores(sentences, slor_scorer, tokenizer, model, verbose = verbose)
      mean_scores = mean_slor(scores)
      result.append(mean_scores)
    except:
      result.append(np.nan)
    
  return result

In [23]:
# this consumes a lot of mem since it loads gpt in the background
# uncomment if you want to see the output
# slor_main(["I have a dog. His name is Jacque", "The sky is blue. The trees are green", "no no jam jam. true ice salad"])
# => [-4.665365032596927, -5.1345871257779825, -8.903362500853792]

## SummaQA

In [13]:
from summ_eval.summa_qa_metric import SummaQAMetric
import numpy as np

global_summaqa = SummaQAMetric()

def get_summaqa_metric(summaries, articles, summaqa=None, aggregate=False):
  if summaqa == None:
    summaqa = SummaQAMetric()
  try:  
    summaqa_dict = summaqa.evaluate_batch(summaries, articles, aggregate = aggregate)
    return summaqa_dict
  except:
    # if we get back this, it likely means memory allocation isn't enough
    # we have run out of memory
    print("Returning NAN value for SummaQA - it is likely that we have reached OOM")
    return {'summaqa_avg_fscore': np.nan, 'summaqa_avg_prob': np.nan}

In [14]:
get_summaqa_metric(refs, articles, global_summaqa)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Returning NAN value for SummaQA - it is likely that we have reached OOM


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


{'summaqa_avg_fscore': nan, 'summaqa_avg_prob': nan}

In [11]:
get_summaqa_metric(predictions, articles, global_summaqa)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Returning NAN value for SummaQA - it is likely that we have reached OOM


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


{'summaqa_avg_fscore': nan, 'summaqa_avg_prob': nan}

In [12]:
get_summaqa_metric(["I like this dog"], ["I like this dog"], global_summaqa)

{'summaqa_avg_fscore': 0.0, 'summaqa_avg_prob': 0.0}

## METEOR

In [18]:
# default version of nltk in colab is 3.2.5 which doesn't support meteor, so upgrade
!pip install -U nltk

     |████████████████████████████████| 1.5 MB 5.0 MB/s 
     |████████████████████████████████| 749 kB 43.2 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [19]:
# https://www.nltk.org/_modules/nltk/translate/meteor_score.html
from nltk.translate import meteor_score
from nltk import word_tokenize
import nltk
import numpy as np

nltk.download('punkt')
nltk.download('wordnet')

# takes in a single prediction and single ref. Both are expected to be strings
def get_meteor_metric(prediction, ref):
  prediction = word_tokenize(prediction)
  ref = [word_tokenize(ref)]
  try:
    return round(meteor_score.meteor_score(ref, prediction), 4)
  except:
    return np.nan

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [20]:
get_meteor_metric(predictions[0], refs[0])

0.1821

In [21]:
get_meteor_metric(predictions[1], refs[1])

0.3701

In [22]:
get_meteor_metric("I like dogs", "I like dogs")

0.9815

In [23]:
get_meteor_metric("I hate dogs", "I like dogs")

0.3333

In [24]:
get_meteor_metric("I dogs like", "I like dogs")

0.5

## CHRF

In [25]:
!pip install sacrebleu

In [26]:
from summ_eval.chrfpp_metric import ChrfppMetric
import sacrebleu
import numpy as np

# original implementation is broken: 
# https://github.com/Yale-LILY/SummEval/blob/master/evaluation/summ_eval/chrfpp_metric.py
# note that this implementation is giving aggregated scores across all references and predictions
def get_chrf_metric(refs, predictions):
  try:
    score = sacrebleu.corpus_chrf(predictions, refs, char_order=6, word_order=0, beta=3, remove_whitespace=False)
    score_dict = {"chrf": score.score}
    return score_dict
  except:
    {'chrf': np.nan}

In [27]:
get_chrf_metric(refs, predictions)

{'chrf': 3.5945363048166783}

In [28]:
get_chrf_metric(["I like dogs"], ["I like dogs"])

{'chrf': 50.0}

## BERT Score

In [11]:
from summ_eval.bert_score_metric import BertScoreMetric
import numpy as np

global_bs = BertScoreMetric()
def get_bert_score_metric(predictions, refs, bert_score=None):
  if bert_score == None:
    bert_score = BertScoreMetric()
  try:
    bs_dict = bert_score.evaluate_batch(predictions, refs)
    return bs_dict
  except:
    return {'bert_score_f1': np.nan, 'bert_score_precision': np.nan, 'bert_score_recall': np.nan}

In [12]:
get_bert_score_metric(predictions, refs, global_bs)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


hash_code: bert-base-uncased_L8_no-idf_version=0.3.10(hug_trans=4.12.5)


{'bert_score_f1': 0.5337008237838745,
 'bert_score_precision': 0.501099169254303,
 'bert_score_recall': 0.5725141763687134}

In [13]:
get_bert_score_metric("I like dogs", "I like dogs", global_bs)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


hash_code: bert-base-uncased_L8_no-idf_version=0.3.10(hug_trans=4.12.5)


{'bert_score_f1': 0.8181817531585693,
 'bert_score_precision': 0.8181817531585693,
 'bert_score_recall': 0.8181817531585693}

## Summary Stats

Here compression is a number representing how compressed the summary is against the original text. Higher number implies more compressed summary (shorter). The higher the compression, the smaller the `summary_length` variable.

Density gives the (loosely) sum of match_length<sup>2</sup> (between summary and context) normalized by the summary length (default) or context length. It is equivalent to coverage.

In [29]:
from summ_eval.data_stats_metric import DataStatsMetric
import numpy as np

def get_data_stats_metric(summaries, articles):
  try:
    ds = DataStatsMetric()
    ds_dict = ds.evaluate_batch(summaries, articles)
    return ds_dict
  except:
    return {'compression': np.nan,
         'coverage': np.nan,
         'density': np.nan,
         'percentage_novel_1-gram': np.nan,
         'percentage_novel_2-gram': np.nan,
         'percentage_novel_3-gram': np.nan,
         'percentage_repeated_1-gram_in_summ': np.nan,
         'percentage_repeated_2-gram_in_summ': np.nan,
         'percentage_repeated_3-gram_in_summ': np.nan,
         'summary_length': np.nan}

def get_repeated_n_gram_in_summ(data_stats_dict, ngram):
  key = "percentage_repeated_{}-gram_in_summ".format(ngram)
  return data_stats_dict[key]

def get_novel_n_gram_in_summ(data_stats_dict, ngram):
  key = "percentage_novel_{}-gram".format(ngram)
  return data_stats_dict[key]

In [30]:
get_data_stats_metric(refs, articles)

Counter({'compression': 24.044471112817885,
         'coverage': 0.7796861665813278,
         'density': 3.1535020908004783,
         'percentage_novel_1-gram': 0.27625287356321837,
         'percentage_novel_2-gram': 0.6200613049189899,
         'percentage_novel_3-gram': 0.7525625422582827,
         'percentage_repeated_1-gram_in_summ': 0.08884072249589492,
         'percentage_repeated_2-gram_in_summ': 0.0058823529411764705,
         'percentage_repeated_3-gram_in_summ': 0.0,
         'summary_length': 29.4})

In [31]:
get_data_stats_metric(predictions, articles)

Counter({'compression': 13.124323445225084,
         'coverage': 0.8998864664262791,
         'density': 3.3680028933249075,
         'percentage_novel_1-gram': 0.14616413314705537,
         'percentage_novel_2-gram': 0.5361024844720497,
         'percentage_novel_3-gram': 0.7272600096170003,
         'percentage_repeated_1-gram_in_summ': 0.373898005852465,
         'percentage_repeated_2-gram_in_summ': 0.18774327122153206,
         'percentage_repeated_3-gram_in_summ': 0.15084852294154621,
         'summary_length': 52.0})

In [32]:
# just for fun - we are typically not supposed to evaluate the prediction against the ref
get_data_stats_metric(predictions, refs)

Counter({'compression': 0.5714516498767084,
         'coverage': 0.3519695674262419,
         'density': 0.5920359218368587,
         'percentage_novel_1-gram': 0.7085461281286708,
         'percentage_novel_2-gram': 0.9193322981366461,
         'percentage_novel_3-gram': 0.972884710539337,
         'percentage_repeated_1-gram_in_summ': 0.373898005852465,
         'percentage_repeated_2-gram_in_summ': 0.18774327122153206,
         'percentage_repeated_3-gram_in_summ': 0.15084852294154621,
         'summary_length': 52.0})

In [33]:
get_novel_n_gram_in_summ(get_data_stats_metric(predictions, articles), 1)

0.14616413314705537

In [34]:
get_repeated_n_gram_in_summ(get_data_stats_metric(predictions, articles), 2)

0.18774327122153206

In [35]:
# interesting that percentage_repeated_1-gram_in_summ is 0 for this example
# also sentence length is 1 which seems interesting - couldn't figure this out reading the code.
# also where are the bi/tri gram stats? needs more investigation
get_data_stats_metric("I love dogs. They are adorable", "I like dogs. They are so cute")

Counter({'compression': 1.0,
         'coverage': 0.6896551724137931,
         'density': 0.6896551724137931,
         'percentage_novel_1-gram': 0.3103448275862069,
         'percentage_repeated_1-gram_in_summ': 0.0,
         'summary_length': 1.0})

## Pyramid Eval


The pyramid method is a manual method for summarization evaluation and it was developed in an attempt to address a key problem in summarization---namely the fact that different humans choose different content when writing summaries. In DUC 2001 to 2004, the manual evaluation was based on comparison with a single human-written model and a lot of the information of evaluated summaries (both human and automic), was marked as "related to the topic, but not directly expressed in the model summary". The pyramid method addresses the problem by using multiple human summaries to create a gold-standard and by expoiting the frequency of information in the human summaries in order to assign importance to different facts.

Info on SCUs: http://www1.cs.columbia.edu/~ani/DUC2005/AnnotationGuide.htm

https://github.com/danieldeutsch/sacrerouge/blob/master/doc/metrics/pyreval.md

We need anaconda for evaluating pyramid metric per the repo above. So we will install mini conda.

In [ ]:
#%cd /content
#!pip install sacrerouge
#!sacrerouge setup-metric pyreval
#!pip install scikit-learn
#!pip install spacy
#import os
#os.environ["CONDA_INIT"] = '/usr/local/bin/conda'

In [ ]:
#%%bash
#MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
#MINICONDA_PREFIX=/usr/local
#wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
#chmod +x $MINICONDA_INSTALLER_SCRIPT
#./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

In [ ]:
#!which conda # should return /usr/local/bin/conda
#!conda create --name pyramidEnv -y

In [ ]:
if False:
  from sacrerouge.metrics import PyrEval
  def get_pyramid_metric(reference, summary):
    # need to pass in the same env name as the one we created in conda
    pyramid = PyrEval(environment_name = 'pyramidEnv')
    return pyramid.score(summary, reference)

In [ ]:
#get_pyramid_metric(refs[0], predictions[0])

## Metrics for Coherence: EGRID, LDA (Topic Coverage)

Taking ideas about coherence scoring from https://www.ijcai.org/Proceedings/05/Papers/0505.pdf

There is a separate notebook that contains all EGRID and coherence metric related outputs

# Main Metric Run

In [24]:
available_metrics = ['rouge', 'rouge_we', 
           'avg_slor', 'summaqa', 'meteor',
           'chrf', 'bert_score', 'summary_stats']
# pyramid eval is broken at the moment - so don't include in metrics list
# EGRID, LDA is implemented in another notebook so join the results from here with that notebook

In [25]:
import pandas as pd
import random
import numpy as np 

def main_metric_run(row):
  assertion = [m in available_metrics for m in metric_list]
  assert sum(assertion) == len(metric_list)

  result_metrics = {}
  for m in metric_list:
    result_metrics[m] = []

  prediction = row['decoded']
  ref = row['reference']
  story_id = row['story_id']
  model_variant = row['model_variant']

  # bizzare behavior in dask where we start processing rows with every col val == foo
  if story_id == 'foo':
    print("row with col values 'foo' detected. This is a weird behavior in dask - row is ignored! \nRow:\n {}".format(row))

  if 'rouge' in metric_list:
    rouge_hf = rouge_scores_hf(predictions = [prediction], refs = [ref])
    
    # no point taking low and high, they will be the same for a single ref/prediction
    rouge_dic = {'1_mid_recall': rouge_hf['rouge1'].mid.recall,
            '2_mid_recall': rouge_hf['rouge2'].mid.recall,
            'L_mid_recall': rouge_hf['rougeL'].mid.recall,
            'Lsum_mid_recall': rouge_hf['rougeLsum'].mid.recall,
            }

    ### CALCULATE WEIGHTED ROUGE BETWEEN SUMMARY AND SENTENCES ###
    # 50-50 weight between sentences and summary importance
    mean_sentences, mean_summ, wt_rouge, median_sentences, min_sentences, max_sentences = get_mid_recall_rouge_stats(rouge_hf, 
                                                                                                                      weights=[0.5,0.5], 
                                                                                                                      verbose=False)
    rouge_dic['recall_mean_sents'] = mean_sentences
    rouge_dic['recall_mean_summ'] = mean_summ
    rouge_dic['recall_median_sents'] = median_sentences
    rouge_dic['recall_min_sents'] = min_sentences
    rouge_dic['recall_max_sents'] = max_sentences
    rouge_dic['wt_sum50_recall'] = wt_rouge

    # 30-70 weight between sentences and summary importance
    mean_sentences, mean_summ, wt_rouge, median_sentences, min_sentences, max_sentences = get_mid_recall_rouge_stats(rouge_hf, 
                                                                                                                      weights=[0.3,0.7], 
                                                                                                                      verbose=False)
    rouge_dic['wt_sum70_recall'] = wt_rouge

    # 70-30 weight between sentences and summary importance
    mean_sentences, mean_summ, wt_rouge, median_sentences, min_sentences, max_sentences = get_mid_recall_rouge_stats(rouge_hf, 
                                                                                                                      weights=[0.7,0.3], 
                                                                                                                      verbose=False)
    rouge_dic['wt_sum30_recall'] = wt_rouge

    ### CALCULATE HIGHER ORDER ROUGE (3/4) ###
    higher_rouge = get_rouge(ngram = 4, prediction = prediction, ref = ref)
    rouge_dic['3_recall'] = higher_rouge['ROUGE-3-R']
    rouge_dic['4_recall'] = higher_rouge['ROUGE-4-R']
    rouge_dic['4_skipgram_recall'] = higher_rouge['ROUGE-SU4-R']
    rouge_dic['4_skipgram_f1'] = higher_rouge['ROUGE-SU4-F']

    result_metrics['rouge'].append(rouge_dic)

  if 'rouge_we' in metric_list:
    rouge_we_dic = {}
    rouge_we_metric1 = rouge_we_metric(refs = [ref], predictions = [prediction], ngram=1)
    rouge_we_dic['1_recall'] = rouge_we_metric1[0]['rouge_we_1_r']
    rouge_we_dic['1_f1'] = rouge_we_metric1[0]['rouge_we_1_f']

    rouge_we_metric2 = rouge_we_metric(refs = [ref], predictions = [prediction], ngram=2)
    rouge_we_dic['2_recall'] = rouge_we_metric2[0]['rouge_we_2_r']
    rouge_we_dic['2_f1'] = rouge_we_metric2[0]['rouge_we_2_f']

    rouge_we_metric3 = rouge_we_metric(refs = [ref], predictions = [prediction], ngram=3)
    rouge_we_dic['3_recall'] = rouge_we_metric3[0]['rouge_we_3_r']
    rouge_we_dic['3_f1'] = rouge_we_metric3[0]['rouge_we_3_f']

    rouge_we_metric4 = rouge_we_metric(refs = [ref], predictions = [prediction], ngram=4)
    rouge_we_dic['4_recall'] = rouge_we_metric4[0]['rouge_we_4_r']
    rouge_we_dic['4_f1'] = rouge_we_metric4[0]['rouge_we_4_f']

    result_metrics['rouge_we'].append(rouge_we_dic)

  if 'meteor' in metric_list:
    meteor_metric = get_meteor_metric(prediction, ref)
    result_metrics['meteor'].append({'metric': meteor_metric})

  if 'chrf' in metric_list:
    chrf_metric = get_chrf_metric([ref], [prediction])
    result_metrics['chrf'].append({'metric': chrf_metric['chrf']})

  if 'summary_stats' in metric_list:
    article = get_cnndm_by_id(dataset, story_id, return_article_only=True)
    if article == None:
      print("Error from summary stats metric. Cannot find story with id: {}.".format(story_id))
      summary_stats_dic = {'compression': np.nan,
                          'coverage': np.nan,
                          'density': np.nan,
                          'pct_novel_1-gram': np.nan,
                          'pct_novel_2-gram': np.nan,
                          'pct_novel_3-gram': np.nan,
                          'pct_repeat_1-gram': np.nan,
                          'pct_repeat_2-gram': np.nan,
                          'pct_repeat_3-gram': np.nan,
                          }
    else:  
      ss_metric = get_data_stats_metric([prediction], [article])
      summary_stats_dic = {'compression': ss_metric['compression'],
                            'coverage': ss_metric['coverage'],
                            'density': ss_metric['density'],
                            'pct_novel_1-gram': ss_metric['percentage_novel_1-gram'],
                            'pct_novel_2-gram': ss_metric['percentage_novel_2-gram'],
                            'pct_novel_3-gram': ss_metric['percentage_novel_3-gram'],
                            'pct_repeat_1-gram': ss_metric['percentage_repeated_1-gram'],
                            'pct_repeat_2-gram': ss_metric['percentage_repeated_2-gram'],
                            'pct_repeat_3-gram': ss_metric['percentage_repeated_3-gram'],
                            }

    result_metrics['summary_stats'].append(summary_stats_dic)
  
  if 'summaqa' in metric_list:
    article = get_cnndm_by_id(dataset, story_id, return_article_only=True)
    if article == None:
      print("Error from summaqa metric. Cannot find story with id: {}.".format(story_id))
      result_metrics['summaqa'].append({'avg_fscore': np.nan, 'avg_prob': np.nan})
    else:  
      summaqa_metric = get_summaqa_metric(summaries = [prediction], articles = [article], summaqa=global_summaqa)
      result_metrics['summaqa'].append({'avg_fscore': summaqa_metric['summaqa_avg_fscore'], 'avg_prob': summaqa_metric['summaqa_avg_prob']})

  if 'bert_score' in metric_list:      
    bert_score = get_bert_score_metric([prediction], [ref], bert_score=global_bs)
    result_metrics['bert_score'].append({'recall': bert_score['bert_score_recall']})

  if 'avg_slor' in metric_list:
    slors = slor_main([prediction, ref])
    result_metrics['avg_slor'].append({'sents_decoded': slors[0], 'sents_reference': slors[1]})
    
  # assign columns the metric values
  for k, res in result_metrics.items():
    cols = [k + '_' + suffix for suffix in res[0].keys()]
    for col in cols:
      row[col] = get_val_for_col(col, res[0]) 
  
  return row

def get_val_for_col(col, dic_row):
  key = [k for k in dic_row.keys() if col.endswith(k)][0]
  return dic_row[key]

In [26]:
%cd /content

/content


In [27]:
import multiprocessing
import numpy as np 
from multiprocessing import Pool
import dask
import pandas as pd
# few parallelization techniques

def parallelize_dataframe(df, func):
    n_cores = multiprocessing.cpu_count()
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

def dask_apply_metric_run(ddata): 
  return ddata.map_partitions(main_metric_run_internal).compute(scheduler='threads') 

def main_metric_run_internal(df): 
  return df.apply((lambda row: main_metric_run(row)), axis=1)

# inspired from here:
# https://examples.dask.org/applications/embarrassingly-parallel.html#Define-the-set-of-input-parameters-to-call-the-function
def dask_lazy_metric_run(df):
  lazy_results = []
  for index, row in df.iterrows():
    lazy_result = dask.delayed(main_metric_run)(row)
    lazy_results.append(lazy_result)
  res = dask.compute(*lazy_results)
  df = pd.concat([r for r in res], axis=1)
  return df.T

In [28]:
import pandas as pd
import dask.dataframe as dd
from dask.multiprocessing import get
import dask

# read df and partition it based on cores available
df = pd.read_csv('data_scored.csv')
#df = df.head(10)

# decide metrics on which to run the df
metric_list = available_metrics 
# be careful when using 'rouge_we' - it takes up a mem - compute in batch mode stand alone
# prefer running 'avg_slor' in batch mode and by itself
# prefer running 'summaqa' and 'bert_score' individually (memory requirements)
metric_list = ['rouge', 'meteor', 'chrf', 'summary_stats']
metric_list = ['bert_score']

# ddf = dd.from_pandas(df, npartitions=5)
# df = dask_apply_metric_run(ddf)
#df = dask_lazy_metric_run(df)

# Store Data as CSV

In [31]:
import datetime
from google.colab import files

def write_to_csv(df, inner_filename):
  now = datetime.datetime.now()
  filename = now.strftime("%Y-%m-%d-%H-%M-%S")

  compression_opts = dict(method='zip', archive_name='{}.csv'.format(inner_filename))

  df.to_csv('{}.zip'.format(filename), index=False, compression = compression_opts)
  files.download('{}.zip'.format(filename))

In [32]:
write_to_csv(df, inner_filename='data_scored')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Extract SummEval Metric Scores

Note that we only extract the 11 reference summary scores for our analysis (from the summ eval paper results)

In [18]:
!pip install gdown
!pip install flatten_json

  Created wheel for flatten-json: filename=flatten_json-0.1.13-py3-none-any.whl size=7980 sha256=f5748fef168679f1b5b0222e7acc41836e927ea9cb97ab473bb0e79e4f506d86
  Stored in directory: /root/.cache/pip/wheels/e6/b3/2a/beb2ceb72d11bf335f9c2f87aae26981f6744f3fc885cde665
Successfully built flatten-json


In [4]:
!gdown https://drive.google.com/uc?id=1d2Iaz3jNraURP1i7CfTqPIj8REZMJ3tS

Downloading...
From: https://drive.google.com/uc?id=1d2Iaz3jNraURP1i7CfTqPIj8REZMJ3tS
To: /content/model_annotations.aligned.scored.jsonl
100% 22.2M/22.2M [00:00<00:00, 104MB/s] 


In [26]:
import numpy as np
import json
import pandas as pd
from flatten_json import flatten

summeval_metric_set = ['summaqa_avg_prob', 'cider', 'percentage_novel_3-gram', 
              'sentence_movers_glove_sms', 'percentage_repeated_3-gram_in_summ', 
              'summary_length', 'compression', 'bert_score_recall', 'rouge_we_2_r', 
              'bleu', 'percentage_novel_2-gram', 'rouge_we_3_r', 
              'rouge_we_2_f', 'coverage', 'rouge_we_3_f', 'density', 
              'supert', 'rouge', 'blanc', 's3_pyr', 'rouge_we_1_p', 
              'percentage_repeated_2-gram_in_summ', 's3_resp', 
              'bert_score_precision', 'rouge_we_1_r', 'rouge_we_1_f', 
              'percentage_repeated_1-gram_in_summ', 'percentage_novel_1-gram', 
              'mover_score', 'chrf', 'summaqa_avg_fscore', 'rouge_we_2_p', 
              'bert_score_f1', 'meteor', 'rouge_we_3_p']

def extract_summeval_metric_scores():
  with open('/content/model_annotations.aligned.scored.jsonl', 'r') as json_file:
    json_list = list(json_file)

    dic_flattened = []
    for json_str in json_list:
        annotation = json.loads(json_str)
        del annotation['metric_scores_1'], annotation['metric_scores_6']
        dic_flattened.append(flatten(annotation))

    data = pd.DataFrame(dic_flattened)
    return data

In [27]:
d = extract_summeval_metric_scores()
write_to_csv(d, inner_filename='summeval_scores')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>